In [1]:
import sys, os
import datetime
import time
import math
sys.path.append(os.path.join(os.getcwd(), '..'))

import talib
import numpy as np
import mplfinance as mpf
import pandas as pd

from lib.dao import event
from lib.dao.data_query import get_ohclv, get_all_pairs
from lib.utils.logger import logger
from lib.notification import send_push
from lib.dao.exchange import exchange, fetch_ticker, load_markets
from lib.utils.time import unify_dt, unify_ts, curr_ts, dt_to_float


2024-05-04 10:42:32,269 - INFO - Enable proxy: http://127.0.0.1:7890


In [2]:
all_pairs = list(filter(lambda pair: pair.endswith('USDT') and pair != 'NBTUSDT', get_all_pairs()))

2024-05-03 17:20:03,952 - INFO - Create an new database session


# 查询过去两天内出现MACD金叉的加密货币加密对


In [7]:
for pair in all_pairs:
    if pair == 'NBTUSDT':
        continue
    
    logger.info(f'Query {pair} in 100 days')
    df = get_ohclv(pair, '1d', limit=100)
    # print(f'==============={pair}================')
    df['macd'], df['macdSignal'], df['macdHist'] = talib.MACD(df['close'])
    if df['macdHist'].iloc[-2] < 0 and df['macdHist'].iloc[-1] >= 0:
        print(f'{pair} 在过去一天出现了金叉')
    if df['macdHist'].iloc[-3] < 0 and df['macdHist'].iloc[-2] >= 0:
        print(f'{pair} 在过去二天出现了金叉')
    # for i in range(len(df)):
    #     if df['macd'].iloc[i-1] < df['macdSignal'].iloc[i-1] and df['macd'].iloc[i] >= df['macdSignal'].iloc[i]:
    #         print(f"{pair} 金叉出现日: {df['timestamp'].iloc[i]}")
    # if df['macdHist'].iloc[-3] < df['macdHist'].iloc[-2] and df['macdHist'].iloc[-2] < df['macdHist'].iloc[-1]:
    #     print(f'{pair} 最近连续三天趋势增强')
    

BTT/USDT 在过去二天出现了金叉
CELR/USDT 在过去一天出现了金叉
USDSB/USDT 在过去一天出现了金叉
TOMO/USDT 在过去一天出现了金叉
RVN/USDT 在过去一天出现了金叉
GXS/USDT 在过去二天出现了金叉
LEND/USDT 在过去一天出现了金叉
JST/USDT 在过去二天出现了金叉
ALPHA/USDT 在过去二天出现了金叉
SXPDOWN/USDT 在过去二天出现了金叉
CTK/USDT 在过去二天出现了金叉
BCHDOWN/USDT 在过去二天出现了金叉
AKRO/USDT 在过去一天出现了金叉
DNT/USDT 在过去一天出现了金叉
AVA/USDT 在过去二天出现了金叉
AAVEDOWN/USDT 在过去二天出现了金叉
BTG/USDT 在过去二天出现了金叉
BURGER/USDT 在过去二天出现了金叉
TRIBE/USDT 在过去一天出现了金叉
OOKI/USDT 在过去二天出现了金叉
T/USDT 在过去一天出现了金叉
NEXO/USDT 在过去一天出现了金叉
HIFI/USDT 在过去一天出现了金叉
SUI/USDT 在过去一天出现了金叉
CYBER/USDT 在过去二天出现了金叉
VIC/USDT 在过去二天出现了金叉
STRK/USDT 在过去二天出现了金叉
ETHFI/USDT 在过去一天出现了金叉


# 监控自选加密货币出现SAR卖点的时刻

In [12]:
mypair = ['BTC/USDT', 'ETH/USDT', 'JTO/USDT', 'DOGE/USDT', 'CTXC/USDT', 'CHZ/USDT' ]
for pair in mypair:
    logger.info(f'Query {pair} in 100 days')
    df = get_ohclv(pair, '1d', limit=100)
    df['sar'] = talib.SAR(df['high'], df['low'], acceleration=0.02, maximum=0.2)
    # # 买入信号：当价格从下方穿过SAR线时
    df.loc[df['close'] > df['sar'], 'temp'] = 1
    # # 卖出信号：当价格从上方穿过SAR线时
    df.loc[df['close'] < df['sar'], 'temp'] = -1
    df['position'] = 0
    df.loc[df['temp'].shift(1) > df['temp'], 'position'] = -1
    df.loc[df['temp'].shift(1) < df['temp'], 'position'] = 1
    # 根据信号列计算仓位状态
    # df['position'] = df['signal'].diff()
    for i in range(1, len(df)):
        if df['position'].iloc[i] < 0:
            print(f'{pair} 在{df["timestamp"].iloc[i]} 出现卖点')

BTC/USDT 在2024-01-22 00:00:00 出现卖点
BTC/USDT 在2024-02-20 00:00:00 出现卖点
BTC/USDT 在2024-03-15 00:00:00 出现卖点
BTC/USDT 在2024-04-02 00:00:00 出现卖点
BTC/USDT 在2024-04-12 00:00:00 出现卖点
ETH/USDT 在2024-03-05 00:00:00 出现卖点
ETH/USDT 在2024-03-15 00:00:00 出现卖点
ETH/USDT 在2024-04-02 00:00:00 出现卖点
ETH/USDT 在2024-04-12 00:00:00 出现卖点
JTO/USDT 在2024-02-01 00:00:00 出现卖点
JTO/USDT 在2024-02-20 00:00:00 出现卖点
JTO/USDT 在2024-02-28 00:00:00 出现卖点
JTO/USDT 在2024-03-19 00:00:00 出现卖点
JTO/USDT 在2024-04-08 00:00:00 出现卖点
DOGE/USDT 在2024-01-23 00:00:00 出现卖点
DOGE/USDT 在2024-03-11 00:00:00 出现卖点
DOGE/USDT 在2024-04-02 00:00:00 出现卖点
CTXC/USDT 在2024-01-22 00:00:00 出现卖点
CTXC/USDT 在2024-02-12 00:00:00 出现卖点
CTXC/USDT 在2024-03-05 00:00:00 出现卖点
CTXC/USDT 在2024-03-13 00:00:00 出现卖点
CTXC/USDT 在2024-04-12 00:00:00 出现卖点
CHZ/USDT 在2024-01-31 00:00:00 出现卖点
CHZ/USDT 在2024-03-05 00:00:00 出现卖点
CHZ/USDT 在2024-03-17 00:00:00 出现卖点
CHZ/USDT 在2024-04-12 00:00:00 出现卖点


# 在过去150天内，MACD 金叉买入，SAR 卖点卖出的收益率

In [50]:
def macd_mix_sar_test(pair):
    pre_price = None
    init_usdt = curr_usdt = 100 # usdt
    curr_btc = 0
    pre_cost = 2 # usdt
    
    df = get_ohclv(pair, '1d', limit=500)
    # 找金叉点
    df['macd'], df['macdSignal'], df['macdHist'] = talib.MACD(df['close'])
    df['temp'] = numpy.nan
    df.loc[df['macdHist'] >= 0, 'temp'] = 1
    df.loc[df['macdHist'] < 0, 'temp'] = -1
    df.loc[df['temp'].shift(1) > df['temp'], 'macd_position'] = -1
    df.loc[df['temp'].shift(1) < df['temp'], 'macd_position'] = 1
    
    # 找SAR卖点
    # TODO: 第一个卖点应该忽略
    df['sar'] = talib.SAR(df['high'], df['low'], acceleration=0.02, maximum=0.2)
    df['temp'] = numpy.nan
    # # 买入信号：当价格从下方穿过SAR线时
    df.loc[df['close'] > df['sar'], 'temp'] = 1
    # # 卖出信号：当价格从上方穿过SAR线时
    df.loc[df['close'] < df['sar'], 'temp'] = -1
    df.loc[df['temp'].shift(1) > df['temp'], 'sar_position'] = -1
    df.loc[df['temp'].shift(1) < df['temp'], 'sar_position'] = 1
   
    for i in range(1, len(df)):
        if df['macd_position'].iloc[i] > 0 and curr_usdt > 0:
            curr_btc = curr_usdt / df['close'].iloc[i]
            pre_cost -= curr_usdt * 0.001
            pre_price = curr_usdt
            curr_usdt = 0
            print(f"{df['timestamp'].iloc[i]} 买入")
            continue
        if df['sar_position'].iloc[i] < 0 and curr_btc > 0:
            curr_usdt = curr_btc * df['close'].iloc[i]
            pre_cost -= (curr_btc * 0.001) * df['close'].iloc[i]
            curr_btc = 0
            rate = (curr_usdt - pre_price)/ pre_price * 100
            print(f"{df['timestamp'].iloc[i]} 卖出, 收益率: {rate}%")
            continue

    final_usdt = (curr_usdt if curr_usdt > 0 else curr_btc * df['close'].iloc[-1]) - (2 - pre_cost) + 2
    final_rate = (final_usdt - (init_usdt + 2)) / (init_usdt + 2) * 100
    print(f'截至最终USDT {final_usdt}， 收益率{final_rate}%')
    
    
    

In [51]:
mypair = ['BTC/USDT', 'ETH/USDT', 'JTO/USDT', 'DOGE/USDT', 'CTXC/USDT', 'CHZ/USDT' ]
for pair in mypair:
    print(f'============{pair}===============')
    macd_mix_sar_test(pair)

============BTC/USDT===============
2024-01-28 00:00:00 买入
2024-02-21 00:00:00 卖出, 收益率: 23.35970113530327%
2024-02-27 00:00:00 买入
2024-03-15 00:00:00 卖出, 收益率: 21.849739135801237%
截至最终USDT 151.8164411557684， 收益率48.83964819192979%
============ETH/USDT===============
2024-01-09 00:00:00 买入
2024-01-22 00:00:00 卖出, 收益率: -1.283544271399876%
2024-02-06 00:00:00 买入
2024-03-05 00:00:00 卖出, 收益率: 49.7761986647785%
2024-04-08 00:00:00 买入
2024-04-12 00:00:00 卖出, 收益率: -12.374242477555166%
2024-04-23 00:00:00 买入
截至最终USDT 134.83938936830774， 收益率32.195479772850724%
============JTO/USDT===============
2024-01-10 00:00:00 买入
2024-01-23 00:00:00 卖出, 收益率: -0.005119017148714988%
2024-01-27 00:00:00 买入
2024-02-23 00:00:00 卖出, 收益率: -4.28119911885835%
2024-02-26 00:00:00 买入
2024-02-28 00:00:00 卖出, 收益率: 1.74178488058898%
2024-03-31 00:00:00 买入
2024-04-08 00:00:00 卖出, 收益率: -1.4324826439816243%
2024-04-21 00:00:00 买入
截至最终USDT 82.10632378687， 收益率-19.5036041305196%
============DOGE/USDT===============
2024-01-16 00

## 如果1h内下跌超过10%的时候买入，最后涨到5%的时候卖出，要等多久

In [3]:
def buy_10_sell_5(pair, draw = False):
    pre_price = None
    init_usdt = curr_usdt = 100 # usdt
    curr_btc = 0
    pre_cost = 2 # usdt
    
    df = get_ohclv(pair, '1h', limit=500)
    # 找下跌10%的点
    buy_points = [np.nan] * len(df)
    sell_points = [np.nan] * len(df)
    df.loc[(df['high'] - df['close']) / df['high'] * 100 > 10, 'position'] = 1

    for i in range(1, len(df)):
        if df['position'].iloc[i] == 1 and curr_usdt > 0:
            # print(df.iloc[i])
            curr_btc = curr_usdt / df['close'].iloc[i]
            pre_cost -= curr_usdt * 0.001
            pre_price = curr_usdt
            curr_usdt = 0
            print(f"{df['timestamp'].iloc[i]} 买入")
            buy_points[i] = df['close'].iloc[i]
            continue
        if curr_btc > 0:
            if (df['close'].iloc[i] * curr_btc - pre_price) / pre_price * 100 > 5:
                # print(df.iloc[i])
                curr_usdt = curr_btc * df['close'].iloc[i]
                pre_cost -= (curr_btc * 0.001) * df['close'].iloc[i]
                curr_btc = 0
                rate = (curr_usdt - pre_price)/ pre_price * 100
                print(f"{df['timestamp'].iloc[i]} 卖出, 收益率: {rate}%")
                sell_points[i] = df['close'].iloc[i]
                continue
    if draw:
        addplot = []
        filter_out_nan = lambda x: x > 0
        if len(list(filter(filter_out_nan, buy_points))) > 0:
            addplot.append(mpf.make_addplot(buy_points, type='scatter', markersize=100, color='y'))
        if len(list(filter(filter_out_nan, sell_points))) > 0:
            addplot.append(mpf.make_addplot(sell_points, type='scatter', markersize=100, color='black'))
        mpf.plot(df.set_index('timestamp'), type='candle', style='yahoo',
             title=f'{pair} Chart', ylabel='Price', ylabel_lower='MACD',
             addplot=addplot,
             figscale=1.5, figsize=(16, 10))
    
    final_usdt = (curr_usdt if curr_usdt > 0 else curr_btc * df['close'].iloc[-1]) - (2 - pre_cost) + 2
    final_rate = (final_usdt - (init_usdt + 2)) / (init_usdt + 2) * 100
    result = {'msg': f'截至最终USDT {final_usdt}， 收益率{final_rate}%', 'rate': final_rate }
    # print(result['msg'])
    return result

In [7]:
results = {}
for pair in all_pairs:
    print(f'============{pair}===============')
    results[pair] = buy_10_sell_5(pair)

2024-04-28 19:54:39,732 - INFO - Create an new database session


============BTC/USDT===============
============ETH/USDT===============
============BNB/USDT===============
============NEO/USDT===============
============LTC/USDT===============
============QTUM/USDT===============
============ADA/USDT===============
2024-04-12 18:00:00 买入
2024-04-12 20:00:00 卖出, 收益率: 6.25669595028927%
============XRP/USDT===============
2024-04-13 20:00:00 买入
2024-04-13 21:00:00 卖出, 收益率: 6.862972408996043%
============EOS/USDT===============
2024-04-12 18:00:00 买入
============TUSD/USDT===============
============IOTA/USDT===============
2024-04-12 18:00:00 买入
2024-04-12 20:00:00 卖出, 收益率: 5.1057040287196%
============XLM/USDT===============
============ONT/USDT===============
============TRX/USDT===============
============ETC/USDT===============
============ICX/USDT===============
2024-04-12 18:00:00 买入
============NULS/USDT===============
2024-04-12 18:00:00 买入
2024-04-12 19:00:00 卖出, 收益率: 5.343511450381655%
============VET/USDT===============
============USDC/USDT

In [ ]:
good = list(filter(lambda x: results[x]['rate'] > 0, results))
bad = list(filter(lambda x: results[x]['rate'] < 0, results))
ok = list(filter(lambda x: results[x]['rate'] == 0, results))
print(len(good), len(bad), len(ok))

In [ ]:
# for pair in bad:
#     buy_10_sell_5(pair, True)

# 获得币对的市值，怀疑这些垃圾币的市值低
# However，市值算不了，没有流通量数据,所以改成查看这些垃圾币发行时间是否比较少，连500都没有
# from lib.dao.exchange import exchange
def get_age(pair):
    return len(get_ohclv(pair, '1d', limit=500))
def less_500_rate(group):
    count_less_500 = 0
    for pair in group:
        age = get_age(pair)
        if age < 500:
            count_less_500 += 1
        # print(f'{pair}: {age}')
    print(f'rate {count_less_500 / len(group) * 100}%')

print(less_500_rate(bad))
print(less_500_rate(good))
print(less_500_rate(ok))

# 找到过去10min内跌幅最大的币对，交易一下

In [4]:
scale = 10
while True:
    curr_time = unify_dt(datetime.datetime.now(), scale * 60)
    messages_10 = []
    messages_5 = []
    decline_rates = []
    for pair in all_pairs:
        df = get_ohclv(pair, '1m', limit=scale + 1)
        
        # for i in range(len(df)):
        max_high = max(df['high'])
        high_idx = 0
        for i in range(len(df)):
            if df['high'].iloc[i] == max_high:
                high_idx = i
                break
    
        data_after_high = df[high_idx:]
        min_low = min(data_after_high['low'])
        min_idx = 0
        for i in range(len(data_after_high)):
            if data_after_high['low'].iloc[i] == min_low:
                min_idx = i
    
        decline_rate = (max_high - min_low) / max_high * 100
        decline_rates.append([pair, decline_rate])
        
        if decline_rate > 10:
            messages_10.append(f'{pair} decline over 10% in {scale}min: {round(decline_rate, 2)}%')
        elif decline_rate > 5:
            messages_5.append(f'{pair} decline over 5% in {scale}min: {round(decline_rate, 2)}%')

    df = pd.DataFrame(decline_rates, columns = ['pair', 'decline_rate'])
    df.sort_values(by='decline_rate', ascending=False, inplace=True)
    print('过去10分钟跌幅前3的交易对: ')
    print(df.head(3))
 
    if len(messages_10) + len(messages_5) > 0:
        messages = messages_10 + messages_5
        print('Send push message:', '\n'.join(messages))
        result = send_push({ 'content': '\n'.join(messages), 'title': f'过去{scale}分钟行情' })
        if not result['success']:
            print('send push failed')

    while unify_dt(datetime.datetime.now(), scale * 60) == curr_time:
        time.sleep(5)

    logger.info('Start next check loop')
    

2024-05-02 09:37:03,823 - INFO - Create an new database session


过去10分钟跌幅前3的交易对: 
          pair  decline_rate
247   VGX/USDT      4.589114
284   REI/USDT      3.924765
281  BIFI/USDT      2.969502


2024-05-02 09:40:02,535 - INFO - Start next check loop


过去10分钟跌幅前3的交易对: 
              pair  decline_rate
247       VGX/USDT      4.180064
550  XVS/USDT:USDT      3.262847
284       REI/USDT      3.257576


KeyboardInterrupt: 

# 海龟交易法测试


In [7]:
def decline_rate(prev, after) -> float:
    return ( after - prev ) / prev * 100
def is_decline_over(prev, after, rate) -> bool:
    if rate == None:
        return False
    if prev == 0:
        return False
    return after < prev and abs(( after - prev ) / prev * 100) > rate
    
def turtle_trade(pair, frame = '1d', limit = 500, max_round = 1, min_window = 20, max_window = 20, stop_decline_rate = None, graph_name = None):
    if limit <= 20:
        raise Exception('Error, days should be at least 21')
    title = f'{pair} {graph_name if graph_name else ""} Chart'
    print(title)
    total_money = 100.0
    cost_rate = 0.001
    hold_coin = 0
    round_number = 0
    
    df = get_ohclv(pair, frame, limit=limit)
    df['max_in_window'] = df['close'].rolling(window=max_window).max()
    df['min_in_window'] = df['close'].rolling(window=min_window).min()
    df['money'] = total_money
    buy_point = [np.nan] * len(df)
    sell_point = [np.nan] * len(df)

    init_coin = (total_money - (total_money * cost_rate)) / df['close'].iloc[0]
    latest_max_price = 0 #买入期间最大的价格

    for i in range(0, len(df)):
        df.loc[i, 'compare'] = df['close'].iloc[i] * init_coin
        
        if i < 20:
            continue
            
        if df['close'].iloc[i] > df['max_in_window'].iloc[i-1] and total_money > 0:
            buy_per_round = total_money / (max_round - round_number)
            round_number += 1
            
            spend_money = 0
            if total_money > buy_per_round:
                spend_money = buy_per_round
                total_money -= buy_per_round
            else:
                spend_money = total_money 
                total_money = 0

            
            hold_coin += (spend_money - (spend_money * cost_rate)) / df['close'].iloc[i]
            if latest_max_price == 0:
                latest_max_price = df['close'].iloc[i]
            buy_point[i] = df['close'].iloc[i]

            print(f"买入价格: {round(df['close'].iloc[i], 4)}, 花费 {round(spend_money, 4)}, 剩余: {round(total_money, 4)}")
        else:
            is_over_min_window = df['close'].iloc[i] < df['min_in_window'].iloc[i-1]
            is_decline_over_threshold = is_decline_over(latest_max_price, df['close'].iloc[i], stop_decline_rate)
            if hold_coin > 0 and (is_over_min_window or is_decline_over_threshold):
                total_money += (hold_coin - (hold_coin * cost_rate)) * df['close'].iloc[i]
                
                message = f"卖出价格: {df['close'].iloc[i]}"
                if is_over_min_window: 
                    message += f" 原因：Over Min Window"
                else:
                    message += f" 原因：Over max threshold from {round(latest_max_price, 4)} to {round(df['close'].iloc[i], 4)} with decline rate {round(decline_rate(latest_max_price, df['close'].iloc[i]), 4)}%"
                print(message)
                
                sell_point[i] = df['close'].iloc[i]
                hold_coin = 0
                latest_max_price = 0
                round_number = 0

        df.loc[i, 'money']= total_money + hold_coin * df['close'].iloc[i]
        if latest_max_price > 0 and df['close'].iloc[i] > latest_max_price:
            latest_max_price = df['close'].iloc[i]
        
    print("最终收益率", round((df['money'].iloc[-1] - df['money'].iloc[0]) / df['money'].iloc[0] * 100, 4), "%")
    # mpf.plot(df.set_index('timestamp'), type='candle', style='yahoo',
    #          title=title, ylabel='Price',
    #          addplot=[
    #              mpf.make_addplot(df['compare'], color='black', panel=1, secondary_y=False),
    #              mpf.make_addplot(df['money'], color='blue', panel=1, secondary_y=False),
    #              mpf.make_addplot(buy_point, type='scatter', markersize=50, color='red', panel=0),
    #              mpf.make_addplot(sell_point, type='scatter', markersize=100, color='green', panel=0)
    #          ],
    #          figscale=1.5, figsize=(16, 10))
    

In [9]:
MONITOR_LIST=[
    'BTC/USDT',
    'ETH/USDT',
    'SOL/USDT',
    'DOGE/USDT',
    'TRB/USDT'
]
for pair in MONITOR_LIST:
    turtle_trade(pair, '1d', 365)
    turtle_trade(pair, '1d', 365, min_window=10, graph_name='Case: decline window is 10')
    turtle_trade(pair, '1d', 365, stop_decline_rate = 10, graph_name='Case: Sell if max decline 10%')
    turtle_trade(pair, '1d', 365, min_window=20, stop_decline_rate = 10, max_round = 2, graph_name='Case: Sell if max decline 10% and buy with 2 round')

BTC/USDT  Chart
买入价格: 28065.0, 花费 100.0, 剩余: 0
卖出价格: 25728.2 原因：Over Min Window
买入价格: 28307.99, 花费 91.4904, 剩余: 0
卖出价格: 29859.13 原因：Over Min Window
买入价格: 27992.57, 花费 96.3107, 剩余: 0
卖出价格: 41732.35 原因：Over Min Window
买入价格: 44349.6, 花费 143.2965, 剩余: 0
卖出价格: 63924.51 原因：Over Min Window
最终收益率 106.1314 %
BTC/USDT Case: decline window is 10 Chart
买入价格: 28065.0, 花费 100.0, 剩余: 0
卖出价格: 25728.2 原因：Over Min Window
买入价格: 28307.99, 花费 91.4904, 剩余: 0
卖出价格: 29895.43 原因：Over Min Window
买入价格: 31454.23, 花费 96.4278, 剩余: 0
卖出价格: 30138.0 原因：Over Min Window
买入价格: 27992.57, 花费 92.208, 剩余: 0
卖出价格: 26875.52 原因：Over Min Window
买入价格: 28500.78, 花费 88.3514, 剩余: 0
卖出价格: 42066.95 原因：Over Min Window
买入价格: 44179.55, 花费 130.1454, 剩余: 0
卖出价格: 42782.73 原因：Over Min Window
买入价格: 44349.6, 花费 125.7787, 剩余: 0
卖出价格: 65300.63 原因：Over Min Window
买入价格: 71620.0, 花费 184.8271, 剩余: 0
卖出价格: 63924.51 原因：Over Min Window
最终收益率 64.6378 %
BTC/USDT Case: Sell if max decline 10% Chart
买入价格: 28065.0, 花费 100.0, 剩余: 0
卖出价格: 25728.2 原因：Over Min 

DODO总是爆拉一波，然后开始跌，策略入场之时就是下跌或暴跌之时，然后因为暴跌就割肉，导致前面出现梯形下降

这个策略，遇到好的行情，会跟进，遇到不好的行情，会及时杀掉，对于那些稳定的币可以采用，长期下跌趋势可以防止损失太大，偶尔涨偶尔跌能够稳住，就怕来个大砸盘。

看起来，某些庄家喜欢等涨20天之后等海龟韭菜们入场来暴跌一波，如果我跌幅的窗口设置小一点，设置成10天呢？-> 相当于沉不住气，容易在一些大震荡行情失利。

这侧策略是否应该设置成下跌到百分之多少就马上割肉？防止大割。 

总结：大好行情，简单海龟交易策略好，割韭菜行情还是要止损

感觉如果涨的太猛也不能入场，因为可能是庄家在爆拉，入场就下跌割肉。

如果觉得自己是在高位买入的话，可以设置一下最大回撤，这样可以追高买入，同时如果遇到大跌可以及时撤回，如果行情继续下跌，就不会蒙受损失。就怕它回涨的幅度和你的最大回撤相同。

轮数设置在2就差不多，设太多容易出现太多的仓位布置在高位。

## 筛选流通性较好的加密货币

In [2]:
def get_good_list(head = 100):
    quote_volumes = []
    all_pairs = list(filter(lambda pair: pair.endswith('USDT') and pair != 'NBTUSDT', get_all_pairs()))
    for pair in all_pairs:
        quote_volumes.append([pair, fetch_ticker(pair)['quoteVolume']])
    df = pd.DataFrame(quote_volumes, columns = ['pair', 'quoteVolume'])
    df.sort_values(by='quoteVolume', ascending=False, inplace=True)
    return df[:head]

In [4]:
res = get_good_list()

In [29]:
all_pairs = list(filter(lambda pair: pair.endswith('USDT') and pair != 'NBTUSDT', get_all_pairs()))

In [12]:
exchange.fetch_ohlcv('BTC/USDT', '1d', limit=20)

[[1713052800000, 63924.52, 65840.0, 62134.0, 65661.84, 61599.17818],
 [1713139200000, 65661.85, 66867.07, 62274.4, 63419.99, 52389.53069],
 [1713225600000, 63419.99, 64365.0, 61600.0, 63793.39, 53435.29331],
 [1713312000000, 63793.4, 64499.0, 59678.16, 61277.37, 50610.54509],
 [1713398400000, 61277.38, 64117.09, 60803.35, 63470.08, 43601.60918],
 [1713484800000, 63470.09, 65450.0, 59600.01, 63818.01, 69774.30271],
 [1713571200000, 63818.01, 65419.0, 63090.07, 64940.59, 23137.42975],
 [1713657600000, 64940.58, 65695.56, 64237.5, 64941.15, 19316.42152],
 [1713744000000, 64941.15, 67232.35, 64500.0, 66819.32, 31397.99371],
 [1713830400000, 66819.32, 67183.01, 65765.81, 66414.0, 22599.90004],
 [1713916800000, 66414.0, 67070.43, 63606.06, 64289.59, 33595.69637],
 [1714003200000, 64289.58, 65297.94, 62794.0, 64498.34, 31341.46338],
 [1714089600000, 64498.33, 64820.01, 63297.48, 63770.01, 27085.19346],
 [1714176000000, 63770.0, 63923.41, 62391.24, 63461.98, 20933.06052],
 [1714262400000, 6346

In [37]:
ts = exchange.fetch_ohlcv('BTC/USDT', '1s', limit=20)[-1][0]

In [40]:
past = unify_ts(ts / 1000, 60 * 60) - unify_ts(ts / 1000, 86400)

In [41]:
past / 3600 

6.0

In [2]:
res = get_ohclv('BTC/USDT', '1s', limit=20)

In [54]:
res['min_in_window'] = res['close'].rolling(window=20).min()

In [55]:
res

,timestamp,open,high,low,close,volume,min_in_window
0,2024-05-03 06:15:32,59477.24,59477.24,59477.24,59477.24,0.04177,NaN
1,2024-05-03 06:15:33,59477.24,59477.24,59477.00,59477.00,1.30497,NaN
2,2024-05-03 06:15:34,59477.01,59477.01,59470.00,59470.00,4.17617,NaN
3,2024-05-03 06:15:35,59470.00,59470.00,59470.00,59470.00,0.00273,NaN
4,2024-05-03 06:15:36,59470.00,59470.00,59470.00,59470.00,0.02252,NaN
5,2024-05-03 06:15:37,59470.00,59470.00,59470.00,59470.00,0.17798,NaN
6,2024-05-03 06:15:38,59470.00,59470.01,59470.00,59470.01,0.34818,NaN
7,2024-05-03 06:15:39,59470.00,59470.01,59470.00,59470.00,0.24296,NaN
8,2024-05-03 06:15:40,59470.00,59470.00,59470.00,59470.00,0.20014,NaN
9,2024-05-03 06:15:41,59470.01,59470.01,59466.43,59466.43,1.26607,NaN


In [3]:
df = get_ohclv('BTC/USDT', '1d', limit=21)

In [4]:
df['timestamp'].iloc[-1].timestamp()

1714694400.0

In [5]:
unify_ts(df['timestamp'].iloc[-1].timestamp(), 86400)

1714694400.0

In [6]:
unify_ts(curr_ts(), 86400) == unify_ts(df['timestamp'].iloc[-1].timestamp(), 86400)

True

In [7]:
df['timestamp'].iloc[-1].to_pydatetime() == unify_dt(datetime.datetime.now(), 86400)

True

In [5]:
df['timestamp'].iloc[-1].to_pydatetime() 

datetime.datetime(2024, 5, 3, 0, 0)

In [6]:
unify_dt(datetime.datetime.now(), 86400)

datetime.datetime(2024, 5, 3, 8, 0)

In [14]:
df['timestamp'].iloc[-1].to_pydatetime()

datetime.datetime(2024, 5, 3, 0, 0)

In [23]:
dt_to_float(datetime.datetime.now())

1714728641.16684

In [7]:
datetime.datetime.utcfromtimestamp(unify_ts(dt_to_float(datetime.datetime.now()), 86400))

datetime.datetime(2024, 5, 3, 0, 0)

In [19]:
unify_dt(datetime.datetime.now(), 86400)

datetime.datetime(2024, 5, 3, 8, 0)

In [20]:
df['timestamp'].iloc[-1].to_pydatetime() 

datetime.datetime(2024, 5, 3, 0, 0)

In [21]:
unify_ts(curr_ts(), 86400) == df['timestamp'].iloc[-1].timestamp()

True